<a href="https://colab.research.google.com/github/joshuaghannan/ECEC247_Project/blob/jgh_tests/colab_version_clean_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### loading modules and data

In [1]:
########################################################

# If running with Google Colab

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# # If using colab
# from google.colab import files
# files.upload()
# # select the 3 .py files (models, utils, data_utils)

In [3]:
########################################################

# If running with Google Colab
# Create a folder "C247" and then store the project datasets within that folder
# Check that your datasets are setup correctly

!ls "/content/gdrive/My Drive/C247" # File path

EEG_loading.ipynb	PyTorch_Experiments.ipynb  X_train_valid.npy
person_test.npy		pytorch_rnn_example.ipynb  y_test.npy
person_train_valid.npy	X_test.npy		   y_train_valid.npy


In [4]:
from models import *
from utils import *
from data_utils import *

GPU is available


In [5]:
# X_test = np.load("X_test.npy")
# y_test = np.load("y_test.npy")
# person_train_valid = np.load("person_train_valid.npy")
# X_train_valid = np.load("X_train_valid.npy")
# y_train_valid = np.load("y_train_valid.npy")
# person_test = np.load("person_test.npy")

# Change if your directory is different

# dataset_path = './data/' # Yiming Path
dataset_path = "/content/gdrive/My Drive/C247/" 

X_test = np.load(dataset_path + "X_test.npy")
y_test = np.load(dataset_path + "y_test.npy")
person_train_valid = np.load(dataset_path + "person_train_valid.npy")
X_train_valid = np.load(dataset_path + "X_train_valid.npy")
y_train_valid = np.load(dataset_path + "y_train_valid.npy")
person_test = np.load(dataset_path + "person_test.npy")
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)


Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


### K-Fold Training and Validation (use k=1 to get results faster)

In [6]:
# setting up the data augmentation here
aug_type = "window"
window_size = 200
vote_num = 20
num_folds = 1
num_epochs = 20

for k in range(num_folds):
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='CNN2LSTM', dropout=0.6, weight_decay=1e-4)
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size)
    if aug_type != 'window':
        X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num)


RNN TYPE: CNN2LSTM
WEIGHT DECAY: 0.0001
LEARNING RATE: 0.001


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.6 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


fold 1
epoch: 1      time: 20.00    loss: 727.171    train acc: 0.292    val acc: 0.355
saving best model...
epoch: 2      time: 19.81    loss: 706.487    train acc: 0.346    val acc: 0.307
epoch: 3      time: 18.47    loss: 668.399    train acc: 0.406    val acc: 0.433
saving best model...
epoch: 4      time: 18.48    loss: 633.331    train acc: 0.455    val acc: 0.456
saving best model...
epoch: 5      time: 18.11    loss: 597.442    train acc: 0.500    val acc: 0.452
epoch: 6      time: 17.16    loss: 555.598    train acc: 0.547    val acc: 0.470
saving best model...
epoch: 7      time: 19.13    loss: 508.113    train acc: 0.595    val acc: 0.444
epoch: 8      time: 18.52    loss: 459.269    train acc: 0.644    val acc: 0.430
epoch: 9      time: 18.37    loss: 408.136    train acc: 0.688    val acc: 0.428
epoch: 10     time: 18.77    loss: 358.782    train acc: 0.731    val acc: 0.442
epoch: 11     time: 18.52    loss: 313.037    train acc: 0.769    val acc: 0.461
epoch: 12     time

### Training and Testing

In [7]:
TestRNN(model, X_test, y_test, person_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

Testing Accuracy: 0.4718


In [0]:
# test

In [0]:
# test1

In [0]:
# test3